# Find K Most Analogous Instances & Precision

#### In order to demonstrate my ability to build things from the scratch and my understanding about topics in data science, I'll implements all algorithm just using numpy and pandas, instead of sklearn and scipy

In [1]:
import pandas as pd
import numpy as np

## Import Data

In [2]:
def make_data(raw):
    data = []
    rows = raw.replace('\n','').split(' ')
    for row in rows:
        data.append(list(map(float,row.split(',')[:4]))+[row[-1]])
    return pd.DataFrame(data)

In [3]:
## Copy and Paste from the pdf file opened in Google Docs
raw = """5.1,3.5,1.4,0.2,A 4.9,3.0,1.4,0.2,A 4.7,3.2,1.3,0.2,A 4.6,3.1,1.5,0.2,A 5.0,3.6,1.4,0.2,A 5.4,3.9,1.7,0.4,A 4.6,3.4,1.4,0.3,A 5.0,3.4,1.5,0.2,A 4.4,2.9,1.4,0.2,A 4.9,3.1,1.5,0.1,A 5.4,3.7,1.5,0.2,A 4.8,3.4,1.6,0.2,A 4.8,3.0,1.4,0.1,A 4.3,3.0,1.1,0.1,A 5.8,4.0,1.2,0.2,A 5.7,4.4,1.5,0.4,A 5.4,3.9,1.3,0.4,A 5.1,3.5,1.4,0.3,A 5.7,3.8,1.7,0.3,A 
5.1,3.8,1.5,0.3,A 5.4,3.4,1.7,0.2,A 5.1,3.7,1.5,0.4,A 4.6,3.6,1.0,0.2,A 5.1,3.3,1.7,0.5,A 4.8,3.4,1.9,0.2,A 5.0,3.0,1.6,0.2,A 5.0,3.4,1.6,0.4,A 5.2,3.5,1.5,0.2,A 5.2,3.4,1.4,0.2,A 4.7,3.2,1.6,0.2,A 4.8,3.1,1.6,0.2,A 5.4,3.4,1.5,0.4,A 5.2,4.1,1.5,0.1,A 5.5,4.2,1.4,0.2,A 4.9,3.1,1.5,0.1,A 5.0,3.2,1.2,0.2,A 5.5,3.5,1.3,0.2,A 4.9,3.1,1.5,0.1,A 
4.4,3.0,1.3,0.2,A 5.1,3.4,1.5,0.2,A 5.0,3.5,1.3,0.3,A 4.5,2.3,1.3,0.3,A 4.4,3.2,1.3,0.2,A 5.0,3.5,1.6,0.6,A 5.1,3.8,1.9,0.4,A 4.8,3.0,1.4,0.3,A 5.1,3.8,1.6,0.2,A 4.6,3.2,1.4,0.2,A 5.3,3.7,1.5,0.2,A 5.0,3.3,1.4,0.2,A 7.0,3.2,4.7,1.4,B 6.4,3.2,4.5,1.5,B 6.9,3.1,4.9,1.5,B 5.5,2.3,4.0,1.3,B 6.5,2.8,4.6,1.5,B 5.7,2.8,4.5,1.3,B 6.3,3.3,4.7,1.6,B 
4.9,2.4,3.3,1.0,B 6.6,2.9,4.6,1.3,B 5.2,2.7,3.9,1.4,B 5.0,2.0,3.5,1.0,B 5.9,3.0,4.2,1.5,B 6.0,2.2,4.0,1.0,B 6.1,2.9,4.7,1.4,B 5.6,2.9,3.6,1.3,B 6.7,3.1,4.4,1.4,B 5.6,3.0,4.5,1.5,B 5.8,2.7,4.1,1.0,B 6.2,2.2,4.5,1.5,B 5.6,2.5,3.9,1.1,B 5.9,3.2,4.8,1.8,B 6.1,2.8,4.0,1.3,B 6.3,2.5,4.9,1.5,B 6.1,2.8,4.7,1.2,B 6.4,2.9,4.3,1.3,B 6.6,3.0,4.4,1.4,B 
6.8,2.8,4.8,1.4,B 6.7,3.0,5.0,1.7,B 6.0,2.9,4.5,1.5,B 5.7,2.6,3.5,1.0,B 5.5,2.4,3.8,1.1,B 5.5,2.4,3.7,1.0,B 5.8,2.7,3.9,1.2,B 6.0,2.7,5.1,1.6,B 5.4,3.0,4.5,1.5,B 6.0,3.4,4.5,1.6,B 6.7,3.1,4.7,1.5,B 6.3,2.3,4.4,1.3,B 5.6,3.0,4.1,1.3,B 5.5,2.5,4.0,1.3,B 5.5,2.6,4.4,1.2,B 6.1,3.0,4.6,1.4,B 5.8,2.6,4.0,1.2,B 5.0,2.3,3.3,1.0,B 5.6,2.7,4.2,1.3,B 
5.7,3.0,4.2,1.2,B 5.7,2.9,4.2,1.3,B 6.2,2.9,4.3,1.3,B 5.1,2.5,3.0,1.1,B 5.7,2.8,4.1,1.3,B 6.3,3.3,6.0,2.5,C 5.8,2.7,5.1,1.9,C 7.1,3.0,5.9,2.1,C 6.3,2.9,5.6,1.8,C 6.5,3.0,5.8,2.2,C 7.6,3.0,6.6,2.1,C 4.9,2.5,4.5,1.7,C 7.3,2.9,6.3,1.8,C 6.7,2.5,5.8,1.8,C 7.2,3.6,6.1,2.5,C 6.5,3.2,5.1,2.0,C 6.4,2.7,5.3,1.9,C 6.8,3.0,5.5,2.1,C 5.7,2.5,5.0,2.0,C 
5.8,2.8,5.1,2.4,C 6.4,3.2,5.3,2.3,C 6.5,3.0,5.5,1.8,C 7.7,3.8,6.7,2.2,C 7.7,2.6,6.9,2.3,C 6.0,2.2,5.0,1.5,C 6.9,3.2,5.7,2.3,C 5.6,2.8,4.9,2.0,C 7.7,2.8,6.7,2.0,C 6.3,2.7,4.9,1.8,C 6.7,3.3,5.7,2.1,C 7.2,3.2,6.0,1.8,C 6.2,2.8,4.8,1.8,C 6.1,3.0,4.9,1.8,C 6.4,2.8,5.6,2.1,C 7.2,3.0,5.8,1.6,C 7.4,2.8,6.1,1.9,C 7.9,3.8,6.4,2.0,C 6.4,2.8,5.6,2.2,C 
6.3,2.8,5.1,1.5,C 6.1,2.6,5.6,1.4,C 7.7,3.0,6.1,2.3,C 6.3,3.4,5.6,2.4,C 6.4,3.1,5.5,1.8,C 6.0,3.0,4.8,1.8,C 6.9,3.1,5.4,2.1,C 6.7,3.1,5.6,2.4,C 6.9,3.1,5.1,2.3,C 5.8,2.7,5.1,1.9,C 6.8,3.2,5.9,2.3,C 6.7,3.3,5.7,2.5,C 6.7,3.0,5.2,2.3,C 6.3,2.5,5.0,1.9,C 6.5,3.0,5.2,2.0,C 6.2,3.4,5.4,2.3,C 5.9,3.0,5.1,1.8,C"""
data = make_data(raw)

In [4]:
data.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,A
1,4.9,3.0,1.4,0.2,A
2,4.7,3.2,1.3,0.2,A
3,4.6,3.1,1.5,0.2,A
4,5.0,3.6,1.4,0.2,A


## Normalization

### Min-Max with Min Substraction

In [5]:
def min_max_min(data):
    X = data.iloc[:,:4]
    df = pd.DataFrame()
    for i in X.columns:
        x_min = np.amin(X.iloc[:,i])
        x_max = np.amax(X.iloc[:,i])
        x = (X.iloc[:,i]-x_min)/(x_max - x_min)
        df = pd.concat([df,x],1)
    df = pd.concat([df,data.iloc[:,4]],1)
    return df

### Min-Max with Mean Substraction

In [6]:
def min_max_mean(data):
    X = data.iloc[:,:4]
    df = pd.DataFrame()
    for i in X.columns:
        x_min = np.amin(X.iloc[:,i])
        x_max = np.amax(X.iloc[:,i])
        x_mean = np.mean(X.iloc[:,i])
        x = (X.iloc[:,i]-x_mean)/(x_max - x_min)
        df = pd.concat([df,x],1)
    df = pd.concat([df,data.iloc[:,4]],1)
    return df

### N(0,1)

In [7]:
def normal(data):
    X = data.iloc[:,:4]
    df = pd.DataFrame()
    for i in X.columns:
        x_mean = np.mean(X.iloc[:,i])
        x_std = np.std(X.iloc[:,i])
        x = (X.iloc[:,i]-x_mean)/x_std
        df = pd.concat([df,x],1)
    df = pd.concat([df,data.iloc[:,4]],1)
    return df

## Distance Metrics

### Euclidean Distance

In [8]:
def euclidean_distance(x,y):
    return np.sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))

### Manhattan Distance

In [9]:
def manhattan_distance(x,y):
    return sum(abs(a-b) for a,b in zip(x,y))

### Minkowski Distance

In [10]:
def minkowski_distance(x,y,order):
    root = 1/float(order)
    return (sum(pow(abs(a-b),order) for a,b in zip(x, y)))**root

## Distance Matrix

In [17]:
def dist_matrix(data, dist_func,order = 200):
    """
    Get pairwise distance among all instances
    Initialize order to be 200 to mimic chebyshev distance
    """
    X = data.iloc[:,:4]
    dist = []
    for i in range(X.shape[0]):
        d=[]
        for j in range(X.shape[0]):
            if dist_func in [euclidean_distance, manhattan_distance]:
                d.append(dist_func(X.iloc[i,:], X.iloc[j,:]))
            else:
                d.append(dist_func(X.iloc[i,:], X.iloc[j,:],order))
        dist.append(d)
    return np.array(dist)

## Find the index of k most similar instances

In [18]:
def k_similar(k,dist_mat):
    """
    Find k most similar instances using sum of k smallest distances
    We iterate the procedure on each row (instance), and update our sum of distance whenever we find a smaller one
    """
    index = None
    dist_sum = np.finfo(float).max
    for i in range(dist_mat.shape[0]):
        ind = np.argsort(dist_mat[i])[:k-1]
        sum_dist = sum(dist_mat[i][ind])
        if sum_dist < dist_sum:
            dist_sum = sum_dist
            index = np.append(ind, i)
    return index

## Precision

In [19]:
def precision(index):
    """
    Define the precision as the percentage of the majority class in the total k instances
    """
    a = 0
    b = 0
    c = 0
    for i in index:
        if data.iloc[i,4] == 'A':
            a += 1
        if data.iloc[i,4] == 'B':
            b += 1
        elif data.iloc[i,4] == 'C':
            c += 1
    k = index.shape[0]
    #print(a,b,c)
    if a>b and a>c:
        return a/k
    if b>a and b>c:
        return b/k
    else:
        return c/k
        

In [20]:
# Load data with 3 different normalization
data_min_max_min = min_max_min(data)
data_min_max_mean = min_max_mean(data)
data_normal = normal(data)

In [22]:
data_normal

,0,1,2,3,4
0,-0.900681,1.032057,-1.341272,-1.312977,A
1,-1.143017,-0.124958,-1.341272,-1.312977,A
2,-1.385353,0.337848,-1.398138,-1.312977,A
3,-1.506521,0.106445,-1.284407,-1.312977,A
4,-1.021849,1.263460,-1.341272,-1.312977,A
5,-0.537178,1.957669,-1.170675,-1.050031,A
6,-1.506521,0.800654,-1.341272,-1.181504,A
7,-1.021849,0.800654,-1.284407,-1.312977,A
8,-1.748856,-0.356361,-1.341272,-1.312977,A
9,-1.143017,0.106445,-1.284407,-1.444450,A


In [21]:
data_loader = [data_min_max_min, data_min_max_mean, data_normal]
dist_loader = [euclidean_distance, manhattan_distance, minkowski_distance]
k_loader = [10,50,100]
data_name = ['data normalized with min_max_min', 'data normalized with min_max_mean', 'data normalized with normal distribution']
dist_name = ['euclidean distance', 'manhattan distance', 'minkowski distance']

for i,df in enumerate(data_loader):
    for j, d in enumerate(dist_loader):
        for k in k_loader:
            d_m = dist_matrix(df,d)
            index = k_similar(k, d_m)
            p = precision(index)
            print('The precision for {} most analogous instances in {} using {} is: {}'.format(k, data_name[i], dist_name[j], p))
            

The precision for 10 most analogous instances in data normalized with min_max_min using euclidean distance is: 1.0
The precision for 50 most analogous instances in data normalized with min_max_min using euclidean distance is: 1.0
The precision for 100 most analogous instances in data normalized with min_max_min using euclidean distance is: 0.5
The precision for 10 most analogous instances in data normalized with min_max_min using manhattan distance is: 1.0
The precision for 50 most analogous instances in data normalized with min_max_min using manhattan distance is: 1.0
The precision for 100 most analogous instances in data normalized with min_max_min using manhattan distance is: 0.5
The precision for 10 most analogous instances in data normalized with min_max_min using minkowski distance is: 1.0
The precision for 50 most analogous instances in data normalized with min_max_min using minkowski distance is: 0.68
The precision for 100 most analogous instances in data normalized with min_ma

There are several insights we can gain from the results. First, for different k, we have different precisions. Larger k will yield lower precision, since it's getting harder to group more instances with the same label into the same group. Then, we can see min_max_min and min_max_mean have similar precisions, but transformation using normal distribution has lower precision. The reason is that first 2 methods compactly transform data into the range (0,1) or (-1,1), but normalization still yield large values since some instances seem to be outliers and the output is just N(0,1), not really squeezed into a range. Thus, the normalized data are still volatile, and distort the relative difference among values for each feature. Finally, let's move on to distance metrics. Manhattan distance and Euclidean distance are both variances of Minkowski distance. Chebyshev distance is when we sum the infinity power of coordinates differences and take infinity squre root. We take 200 for the purpose of demonstration. Clearly, Chebyshev distance doesn't perform well because it only cares about the maximum distance along some axis among all axes. Euclidean and Manhattan distance don't show much difference in this dataset and we can examine their influence on precision on a larger dataset if available.